In [1]:
import pandas as pd
import numpy as np

import json
import spacy

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
pd.set_option('display.max_rows', 200_000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 2_500)

In [3]:
files = ['debate_globo_1.txt', 'debate_globo_2.txt', 'debate_globo_3.txt', 'debate_globo_4.txt', 'debate_globo_5.txt', 'debate_globo_6.txt']

data = []
for file in files:
    for line in open(file, 'r'):
        if line != '\n':
            data.append(json.loads(line))

In [4]:
len(data)

548660

In [5]:
data = [line for line in data if 'created_at' in line]

In [6]:
len(data)

508503

In [9]:
data[0]

{'created_at': 'Fri Oct 05 00:51:03 +0000 2018',
 'id': 1048012658304323589,
 'id_str': '1048012658304323589',
 'text': 'Nada de glória a Deux hoje af\n\n"TSE nega pedido, e Daciolo não poderá participar de debate"  https://t.co/Cnt0Y7er1C',
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 939191085464768512,
  'id_str': '939191085464768512',
  'name': 'yuni sato #12',
  'screen_name': 'yunisatto',
  'location': 'Mato Grosso do Sul, Brasil',
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 2665,
  'friends_count': 2633,
  'listed_count': 0,
  'favourites_count': 16397,
  'statuses_count': 3947,
  'created_at': 'Fri Dec 08 17:52:58 +0000 2017',
  'utc_offset

In [18]:
tweets_list = []
datetime = []

for tweet in data:
    date = pd.Timestamp(tweet['created_at'])
    datetime.append(date + pd.Timedelta(hours=-3))
    
    try:
        try:
            if tweet['text'].startswith('RT'):
                try:
                    tweets_list.append(tweet['retweeted_status']['extended_tweet']['full_text'])
                except:
                    tweets_list.append(tweet['retweeted_status']['text'])
            else:
                tweets_list.append(tweet['text'])
        except:
            tweets_list.append(tweet['extended_tweet']['full_text'])
    except:
        tweets_list.append(tweet['text'])

In [19]:
tweets = pd.DataFrame({'text': tweets_list, 'datetime': datetime})

tweets.head()

,text,datetime
0,"Nada de glória a Deux hoje af\n\n""TSE nega pedido, e Daciolo não poderá participar de debate"" https://t.co/Cnt0Y7er1C",2018-10-04 21:51:03+00:00
1,Pra todo mundo que disse q vai p fora do Brasil se o bolsonaro ganhar eu tenho uma pergunta pra vcs...\nJá compararam as passagens ? KKKK,2018-10-04 21:51:03+00:00
2,e desde quando o ciro gomes tá preso? vote ciro 12 https://t.co/9XjASmAxDv,2018-10-04 21:51:03+00:00
3,"kd meu mosaico na tv com o debate, cruzeiro, nhl, nfl, mlb e de ferias com o ex ao mesmo tempo",2018-10-04 21:51:03+00:00
4,"Se dependesse deste candidato e seu filho, 45 milhões de brasileiros não teriam sido beneficiados pela Lei Brasileira de Inclusão. #HaddadÉ13\n\nEntenda: https://t.co/yxPyG9C6OG https://t.co/cFRmOwYbvf",2018-10-04 21:51:03+00:00


In [20]:
tweets.shape

(508503, 2)

In [21]:
#tweets.to_csv('tweets_debate_globo.csv', index=False)

In [22]:
from spacy.lang.pt.stop_words import STOP_WORDS

D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
D:\Anaconda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [23]:
import string
SYMBOLS = ' '.join(string.punctuation).split(' ') + ['-', '...', '”', '”']

In [25]:
nlp = spacy.load('pt')

In [27]:
import re

def normalizar(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    clean_comment = re.sub(r'\bhttps://t\.co/\w+\b', '', comment)
    clean_comment = nlp(clean_comment)
    lemmatized = list()
    for word in clean_comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and ((lemma not in STOP_WORDS) and (lemma not in SYMBOLS))):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

In [28]:
tweets['text_clean'] = tweets['text'].apply(normalizar, lowercase=True, remove_stopwords=True)

In [29]:
tweets_shuffled = tweets.sample(frac=1).reset_index(drop=True)

In [30]:
tweets_shuffled.to_csv('tweets_debate_globo_limpo.csv', index=False)

In [64]:
tweets_clean = pd.read_csv('tweets_debate_globo_limpo.csv')

tweets_clean.head()

,text,datetime,text_clean
0,"Boulos disse o essencial, que ninguem está dizendo: a democracia está em perigo. Isso é sério pra caralho.",2018-10-04 22:42:25+00:00,boulos o essencial ninguem o democracia perigar sério pra caralho
1,"Debate nada, vou é assistir Netflix",2018-10-04 23:17:46+00:00,debater nado assistir netflix
2,Boulos: A\r\nEu: CONTE COMIGO PRA TUDO (nas próximas eleições),2018-10-04 23:19:00+00:00,boulos o contar comigo pra o eleição
3,"O microfone do Meirelles caiu agora.\r\n\r\nO ponto, já faz uns 30 anos.",2018-10-04 22:27:35+00:00,o microfone meirelles cair o pontar 30
4,"PROPOSTAS DE BOLSONARO\r\n- Fim do 13º e férias\r\n- Redução da licença maternidade\r\n- IR igual pra ricos e POBRES\r\n- Volta da CPMF\r\n- Fim do Bolsa Família e projetos sociais\r\n- Entrega da Petrobrás,BB e Caixa para americanos\r\n- Mais armas em circulação\r\n- Teto de gastos com saúde e educação",2018-10-04 22:31:34+00:00,proposto bolsonaro 13º e féria redução licença maternidade igual pra rico e pobre voltar cpmf bolsar família e projetos social entregar petrobrás b e caixa parir americano armar circulação teto gasto saudar e educação


In [65]:
tweets_clean = tweets_clean[tweets_clean['text_clean'].notnull()].copy().reset_index(drop=True)

In [125]:
sentiments_list = [0, -1, 1, -1, -1, 0, -1, 1, 0, 0, #0xx
                   1, -1, -1, -1, 0, -1, 0, 1, 1, 0,
                   0, -1, -1, 1, 0, 1, 1, 1, -1, 1,
                   1, 0, 1, 0, 1, -1, -1, -1, -1, -1,
                   1, -1, 1, -1, -1, 0, 0, -1, -1, -1,
                   0, -1, 1, 0, 0, -1, -1, -1, -1, 1,
                   -1, -1, 0, -1, -1, -1, -1, -1, -1, 1,
                   1, 1, 1, -1, -1, 1, 1, -1, -1, -1,
                   1, 1, -1, -1, 0, 1, 1, -1, 1, 0,
                   1, 1, -1, -1, -1, -1, -1, 1, 1, 1,
                   1, 1, -1, -1, -1, 0, -1, 1, -1, -1, #1xx
                   -1, 0, -1, 1, -1, 1, 0, -1, -1, -1,
                   -1, -1, -1, 0, 0, 0, -1, -1, 0, -1,
                   1, 1, 0, -1, -1, 1, 1, 1, 1, 0,
                   -1, 1, 0, -1, -1, -1, 0, -1, 0, 1,
                   1, -1, 0, 1, 1, -1, 0, 0, -1, -1,
                   1, 1, 1, 0, 1, 1, 1, 0, -1, 1,
                   1, 1, 0, 0, -1, -1, -1, 0, -1, -1,
                   0, -1, 0, 1, -1, -1, -1, 0, 0, 0,
                   0, -1, 1, 1, 0, -1, 0, -1, 1, -1,
                   1, 1, 0, 1, 1, 1, 0, 0, -1, 1,   #2xx
                   0, 1, 0, -1, 0, -1, -1, 0, -1, 0,
                   0, -1, 1, 0, -1, 0, 1, 1, 1, 0,
                   1, 1, 1, -1, -1, 1, 1, -1, -1, 1,
                   1, 1, 0, 1, 0, 1, 1, -1, -1, 1,
                   0, -1, 0, -1, 0, -1, 1, 0, -1, 1,
                   0, 1, -1, 0, 0, 1, 1, 0, 0, -1,
                   0, 0, 0, 1, -1, 1, 0, 1, 1, 0,
                   -1, 1, 0, 1, 1, 1, -1, 1, 1, -1,
                   0, 1, 1, 0, -1, -1, 0, -1, -1, 0,
                   -1, 1, 0, -1, 1, 1, 1, -1, -1, -1,  #3xx
                   -1, 0, 1, -1, 1, 0, 0, -1, 1, 1,
                   -1, 0, 1, 1, -1, 1, -1, 1, 1, 0,
                   -1, -1, -1, 0, -1, 1, 1, 1, 1, -1,
                   -1, -1, 1, 0, -1, -1, 1, 1, -1, 1,
                   1, -1, 1, -1, -1, -1, 1, -1, 0, 1,
                   -1, 1, -1, 0, -1, 1, 1, -1, 1, 1,
                   -1, 0, 0, -1, 0, -1, -1, 1, -1, 1,
                   1, 0, 0, 1, 0, 0, -1, -1, 1, 1,
                   -1, -1, -1, 1, -1, -1, -1, 1, 0, 0,
                   -1, 1, -1, -1, 1, -1, -1, 0, 0, 1,  #4xx
                   0, 1, 0, 0, 1, 0, 0, -1, 1, -1,
                   1, 0, 0, 1, 1, 0, -1, 0, -1, 1,
                   1, 1, 1, 1, -1, 1, -1, 1, -1, -1,
                   -1, 0, 1, 0, 1, -1, -1, -1, -1, 0,
                   1, 1, 1, 0, 0, 0, 1, -1, -1, -1,
                   1, -1, -1, 1, 0, -1, 0, 1, -1, -1,
                   -1, -1, 0, 1, -1, 1, 0, -1, -1, 0, 
                   1, 0, 1, 1, 0, 0, -1, 1, 1, 0,
                   1, -1, -1, 0, -1, 1, 0, 1, -1, -1,
                   -1, 1, -1, 0, 1, -1, 0, -1, 0, -1,  #5xx
                   -1, 0, -1, -1, -1, -1, -1, 0, 0, -1,
                   0, 0, 0, 0, 0, 1, -1, 1, -1, 0,
                   1, -1, -1, 0, -1, -1, 0, -1, 1, -1,
                   -1, -1, 1, -1, -1, 1, 1, 0, 1, -1,
                   -1, -1, 1, 1, -1, -1, -1, 1, 0, -1,
                   1, -1, -1, -1, 1, 1, -1, 1, -1, -1,
                   -1, 1, -1, 1, -1, 0, -1, 0, -1, 1,
                   -1, 1, -1, -1, 0, 1, 0, 1, 0, 1,
                   -1, 1, -1, 1, 1, 0, 1, 0, -1, 0,
                   1, -1, -1, -1, 1, -1, -1, 0, -1, 1,  #6xx
                   1, 0, -1, 1, 1, 0, -1, -1, 1, -1,
                   1, -1, 0, 1, -1, 0, 1, 1, 1, 0,
                   1, -1, 0, 1, 0, 1, 1, 1, -1, 1,
                   0, 1, -1, 1, 1, -1, 1, 1, 0, -1,
                   1, -1, -1, 1, 1, -1, -1, -1, 0, 1,
                   1, 1, 1, -1, -1, 1, -1, 1, 0, 1,
                   0, 1, 0, 1, -1, -1, 0, -1, 0, -1,
                   -1, -1, -1, 1, 0, -1, -1, 1, 0, 1,
                   1, -1, -1, 1, -1, -1, -1, -1, -1, 1,
                   -1, -1, -1, 1, 1, 1, -1, 1, -1, 1,   #7xx
                   -1, -1, -1, 1, 1, 1, 1, 1, 0, -1,
                   0, 0, -1, -1, 0, -1, 1, 0, 1, 1,
                   -1, 1, 1, 1, 1, 1, -1, 1, 1, -1,
                   -1, -1, -1, 1, -1, -1, -1, 0, -1, 1,
                   1, 0, -1, -1, -1, -1, 1, -1, 0, 1,
                   1, -1, 1, -1, 1, 1, -1, -1, 1, -1,
                   1, 0, -1, -1, 0, -1, -1, -1, 0, -1,
                   0, 1, 1, -1, 0, -1, -1, -1, -1, 1,
                   -1, 0, -1, 1, 1, 1, 1, 0, -1, 0,
                   0, 1, -1, 1, 1, -1, 0, 0, 0, 1,  #8xx
                   1, 1, -1, -1, 1, 0, 0, -1, 1, 1,
                   -1, 1, 0, -1, -1, -1, -1, 1, 0, 1,
                   -1, 0, 1, 1, -1, -1, 0, 1, 1, 1,
                   -1, -1, -1, 1, -1, -1, -1, 0, -1, 1,
                   -1, -1, 1, 1, -1, 1, -1, -1, -1, 1,
                   1, -1, 0, 1, 1, -1, 1, 1, 0, 1,
                   1, -1, -1, 1, 1, -1, -1, -1, 0, 1,
                   -1, 1, 0, 1, 1, -1, 1, 1, -1, 1,
                   -1, -1, 0, -1, 0, 1, -1, -1, 0, -1,
                   0, -1, -1, 0, -1, -1, 0, 0, 1, 1,  #9xx
                   1, -1, -1, -1, 1, 1, -1, -1, 0, 0,
                   -1, 1, 1, 0, 1, 1, 0, 1, 0, 0,
                   0, 0, 0, 0, -1, 1, -1, -1, 1, 1,
                   -1, 1, 0, 1, 1, 1, 1, 0, 0, 0,
                   1, -1, -1, 1, -1, 1, -1, 1, 1, 0,
                   0, 0, 0, -1, -1, 1, 1, 1, 1, 0,
                   1, -1, 0, -1, 0, -1, 0, 1, 1, -1,
                   1, -1, 0, 0, 1, 0, 1, -1, 1, -1,
                   0, 1, -1, 0, -1, 0, -1, -1, -1, 1]

In [126]:
len(sentiments_list)

1000

In [129]:
sentiments = pd.Series(sentiments_list)

tweets_clean['sentiment'] = sentiments

tweets_clean.head()

,text,datetime,text_clean,sentiment
0,"Boulos disse o essencial, que ninguem está dizendo: a democracia está em perigo. Isso é sério pra caralho.",2018-10-04 22:42:25+00:00,boulos o essencial ninguem o democracia perigar sério pra caralho,0.0
1,"Debate nada, vou é assistir Netflix",2018-10-04 23:17:46+00:00,debater nado assistir netflix,-1.0
2,Boulos: A\r\nEu: CONTE COMIGO PRA TUDO (nas próximas eleições),2018-10-04 23:19:00+00:00,boulos o contar comigo pra o eleição,1.0
3,"O microfone do Meirelles caiu agora.\r\n\r\nO ponto, já faz uns 30 anos.",2018-10-04 22:27:35+00:00,o microfone meirelles cair o pontar 30,-1.0
4,"PROPOSTAS DE BOLSONARO\r\n- Fim do 13º e férias\r\n- Redução da licença maternidade\r\n- IR igual pra ricos e POBRES\r\n- Volta da CPMF\r\n- Fim do Bolsa Família e projetos sociais\r\n- Entrega da Petrobrás,BB e Caixa para americanos\r\n- Mais armas em circulação\r\n- Teto de gastos com saúde e educação",2018-10-04 22:31:34+00:00,proposto bolsonaro 13º e féria redução licença maternidade igual pra rico e pobre voltar cpmf bolsar família e projetos social entregar petrobrás b e caixa parir americano armar circulação teto gasto saudar e educação,-1.0


In [130]:
train = tweets_clean[tweets_clean['sentiment'].notnull()]

holdout = tweets_clean[tweets_clean['sentiment'].isnull()].drop('sentiment', axis=1)

train.to_csv('tweets_train.csv', index=False)
holdout.to_csv('tweets_holdout.csv', index=False)